In [1]:
%%time
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from geopy.distance import geodesic as gd
import datetime as dt
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
from IPython.display import clear_output
from scipy.spatial import cKDTree

# loading oil trejectory geojson data using geopandas
oil_path = gpd.read_file('./data/TrajRev-d30-p100-r0.005.geojson')

# loading ship location csv file and converting to geodataframe
ship_data = pd.read_csv('./data/contacts_nt_ais_s_area_SAR_bras_dec21_stream5.csv')
ship_gdf = gpd.GeoDataFrame(ship_data, crs='EPSG:4326', geometry=gpd.points_from_xy(ship_data['lon'], ship_data['lat']))
ship_gdf['dh'] = pd.to_datetime(ship_gdf['dh'], format="%d-%m-%y %H:%M")

def oil_tracker(selected_day, buffer_size, time_interval):
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(14, 8))
    m = Basemap(projection='merc', ax=ax, llcrnrlat=-8, urcrnrlat=-5, llcrnrlon=-35, urcrnrlon=-30, epsg=4326, resolution='l')

    br_admin = gpd.read_file('./data/BR_Municipios_2020.shp')
    br_admin_filtered = br_admin.cx[-35:-30, -8:-5]
    br_admin_filtered.plot(ax=ax, color="white", linewidth=1, edgecolor='#c3c3d6')
    
    m.drawparallels(np.arange(-8, -5, 0.5), labels=[1, 0, 0, 0], fontsize=10, labelstyle='+/-', linewidth=0.1)
    m.drawmeridians(np.arange(-35, -30, 1), labels=[0, 0, 0, 1], fontsize=10, labelstyle='+/-', linewidth=0.1)

    ## Calculating buffer size in degrees
    buffer_size = round(buffer_size/111, 2)

    ## creating buffer areas/zones
    tra_buffer = oil_path['geometry'].buffer(buffer_size, resolution=16)
    tra_buffer_gdf = gpd.GeoDataFrame(geometry=tra_buffer, crs='EPSG:4326')

    oil_filtered = oil_path[oil_path['time'].dt.day == selected_day].copy()
    lon, lat = oil_filtered['geometry'].x, oil_filtered['geometry'].y
    center_point = Point(lon.mean(), lat.mean())
    buffered_area = center_point.buffer(buffer_size, resolution=16)
    buffer_area_gdf = gpd.GeoDataFrame(geometry=[buffered_area], crs='EPSG:4326')

    # Merging spatial datasets sjoins
    within_buffered_area = gpd.sjoin(ship_gdf, tra_buffer_gdf, how='inner', predicate='within')
    within_buffered_area['day'] = within_buffered_area['dh'].dt.day
    oil_path['day'] = oil_path['time'].dt.day

    time_interval = pd.Timedelta(minutes=time_interval)
    reference_times = oil_path.groupby('day')['time'].min()
    filtered_dfs = within_buffered_area.groupby('day').apply(
        lambda df: df[(df['dh'] >= reference_times.loc[df['day'].iloc[0]] - time_interval / 2) &
                      (df['dh'] < reference_times.loc[df['day'].iloc[0]] + time_interval / 2)]
    ).drop(['index_right'], axis=1)

    within_buffered_day = gpd.sjoin(filtered_dfs, buffer_area_gdf, how='inner', predicate='within')

    nA = np.array(list(filtered_dfs.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(oil_path.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    distance, idx = btree.query(nA, k=1)
    oil_path_nearest = oil_path.iloc[idx.squeeze()].reset_index(drop=True)
    filtered_dfs['distance'] = distance.squeeze()
    filtered_dfs['distance'] = round(filtered_dfs['distance'] * 111, 2)
    
    oil_path['day'] = oil_path['time'].dt.day
    oil_centroids = oil_path.groupby('day')['geometry'].apply(lambda x: MultiPoint(x.tolist()).centroid)

    # Calculate the time difference between the ship's timestamp and the oil spill timestamp
    oil_times = oil_path.groupby('day')['time'].min()
    filtered_dfs['time_diff'] = filtered_dfs.apply(lambda row: (row['dh'] - oil_times[row['day']]).total_seconds() / 60, axis=1)  # time difference in minutes
    unique_filtered_dfs = filtered_dfs.drop_duplicates()

    # Ranking ships in time and distance
    normalized_distance = unique_filtered_dfs['distance'] / unique_filtered_dfs['distance'].max()
    normalized_time = abs(unique_filtered_dfs['time_diff']) / abs(unique_filtered_dfs['time_diff']).max()
    unique_filtered_dfs = unique_filtered_dfs.copy()
    unique_filtered_dfs['Score'] = (1 + 99 * (normalized_distance + normalized_time)).astype(int)
    lowest_rank_indices = unique_filtered_dfs.groupby(['mmsi', 'nome_navio'])['Score'].idxmin()
    unique_filtered_dfs = unique_filtered_dfs.loc[lowest_rank_indices]
    unique_filtered_dfs['Ship_rank'] = unique_filtered_dfs['Score'].rank(method='min').astype(int)
    unique_filtered_dfs.sort_values(by='Ship_rank', ascending=True, inplace=True)
    ship_ranking = unique_filtered_dfs
    # ship_ranking.to_csv('./ranking.csv', index=False)

    ### Plotting data to map
    start_date = oil_path['time'].min()
    end_date = oil_path['time'].max()
    ax.set_title(f'AIS Stream5 - {start_date.strftime("%d-%b-%Y")} to {end_date.strftime("%d-%b-%Y")}', fontsize=10)

    ax.set_xlabel('Longitude', labelpad=20, fontsize=10)
    ax.set_ylabel('Latitude', labelpad=40, fontsize=10)

    tra_buffer.plot(ax=ax, linewidth=2)
    oil_path.plot(ax=ax, c=oil_path['time'].dt.day)
    ship_ranking.plot(ax=ax, c='orange', marker='o', markersize=25)
    buffer_area_gdf.plot(ax=ax, color='blue', edgecolor='k', linewidth=2, alpha=0.5)
    oil_filtered.plot(ax=ax, c='red', markersize=8)

    return ship_ranking.drop(['geometry', 'day'], axis=1)
plt.show()

buffer_slider = widgets.SelectionSlider( options=[5, 10, 15, 20, 30,40, 50, 60], value=10, description='Space (km) ', layout=Layout(width='400px'))
time_slider = widgets.IntSlider(min=0, max=180, step=15, description='Time (min):', value=30, layout=Layout(width='400px'))
day_slider = widgets.IntSlider(min=1, max=max(oil_path['time'].dt.day), step=1, description='Day:', value=8, layout=Layout(width='400px'))
day_slider.style.handle_color, time_slider.style.handle_color, buffer_slider.style.handle_color  = 'red', 'gold', 'green'
oil_tracker_widget = widgets.interact(oil_tracker, buffer_size=buffer_slider, time_interval=time_slider, selected_day=day_slider)

interactive(children=(IntSlider(value=8, description='Day:', layout=Layout(width='400px'), max=31, min=1, styl…

CPU times: total: 16 s
Wall time: 38.1 s
